In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [4]:
##CREATION OF DIFFERENT RECOMMENDERS TUNED

### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_stacked)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

In [5]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [14:13<00:00, 35.70it/s]

In [14]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=int(43.58540579197548), num_factors=int(50.69325524471134), alpha=0.7390441426189945, reg=4.874312446539656)

IALSRecommenderLinear: Epoch 1 of 43. Elapsed time 10.27 sec
IALSRecommenderLinear: Epoch 2 of 43. Elapsed time 19.96 sec
IALSRecommenderLinear: Epoch 3 of 43. Elapsed time 28.92 sec
IALSRecommenderLinear: Epoch 4 of 43. Elapsed time 38.59 sec
IALSRecommenderLinear: Epoch 5 of 43. Elapsed time 48.43 sec
IALSRecommenderLinear: Epoch 6 of 43. Elapsed time 1.01 min
IALSRecommenderLinear: Epoch 7 of 43. Elapsed time 1.21 min
IALSRecommenderLinear: Epoch 8 of 43. Elapsed time 1.41 min
IALSRecommenderLinear: Epoch 9 of 43. Elapsed time 1.61 min
IALSRecommenderLinear: Epoch 10 of 43. Elapsed time 1.82 min
IALSRecommenderLinear: Epoch 11 of 43. Elapsed time 2.02 min
IALSRecommenderLinear: Epoch 12 of 43. Elapsed time 2.24 min
IALSRecommenderLinear: Epoch 13 of 43. Elapsed time 2.44 min
IALSRecommenderLinear: Epoch 14 of 43. Elapsed time 2.65 min
IALSRecommenderLinear: Epoch 15 of 43. Elapsed time 2.90 min
IALSRecommenderLinear: Epoch 16 of 43. Elapsed time 3.11 min
IALSRecommenderLinear: Epoch

In [8]:
##UserKNNCF
UserKNNCF_recommender = UserKNNCFRecommender(URM_train = URM_stacked)
UserKNNCF_recommender.fit(topK=450, shrink=5.0, similarity="tanimoto")

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13984 (100.0%), 743.20 column/sec. Elapsed time 18.82 sec


In [9]:
##PURE_SVD
PURE_SVD = PureSVDItemRecommender(URM_stacked)
PURE_SVD.fit(num_factors=23,topK=561)

PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!


In [19]:
USERCF_PURESVD=MergeTwoModelsByScores(URM_train=URM_stacked, recommenders=[UserKNNCF_recommender,PURE_SVD])
USERCF_PURESVD.fit(alpha=0.14062495687439416, beta=1.0)

In [20]:
USERCF_PURESVD_SLIME=MergeTwoModelsByScores(URM_train=URM_stacked, recommenders=[USERCF_PURESVD,SLIME_recommender])
USERCF_PURESVD_SLIME.fit(alpha=0.08057891449236342, beta=0.557107802182469)

In [21]:
USERCF_PURESVD_SLIME_IALS=MergeTwoModelsByScores(URM_train=URM_stacked, recommenders=[USERCF_PURESVD_SLIME,IALS_recommender])
USERCF_PURESVD_SLIME_IALS.fit(alpha=0.7415216864386212, beta=0.08416474584041955)

In [22]:
USERCF_PURESVD_SLIME_IALS_RP3=MergeTwoModelsByScores(URM_train=URM_stacked, recommenders=[USERCF_PURESVD_SLIME_IALS,RP3Beta_recommender])
USERCF_PURESVD_SLIME_IALS_RP3.fit(alpha=0.7415796670173784, beta=0.16018156909656459)

In [23]:
submission = create_submission(USERCF_PURESVD_SLIME_IALS_RP3)
write_submission(submission,"USERCF_PURESVD_SLIME_IALS_RP3")

100%|████████████████████████████████████▉| 18056/18059 [39:50<00:00,  7.55it/s]
